In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
import os

In [4]:
URL = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
zip_file = tf.keras.utils.get_file(origin = URL, fname = 'flower_photos.tgz', extract = True, cache_subdir = '/content')
base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

228818944/228813984 [==============================] - 159s 1us/step


In [5]:
print(os.path)

<module 'ntpath' from 'D:\\ProgramData\\Anaconda3\\lib\\ntpath.py'>


In [6]:
print(base_dir)

C:/content\flower_photos


In [7]:
IMAGE_SIZE = 224
BATCH_SIZE = 64
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255, validation_split = 0.2)
train_generator = datagen.flow_from_directory(base_dir, target_size = (IMAGE_SIZE, IMAGE_SIZE), batch_size = BATCH_SIZE, subset = 'training')
val_generator = datagen.flow_from_directory(base_dir, target_size = (IMAGE_SIZE, IMAGE_SIZE), batch_size = BATCH_SIZE, subset = 'validation')

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [8]:
print(train_generator.class_indices)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
    f.write(labels)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [9]:
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape = IMAGE_SHAPE, include_top = False, weights = 'imagenet')

In [10]:
base_model.trainable = False

In [11]:
model = tf.keras.Sequential([base_model, tf.keras.layers.Conv2D(32, 3, activation = 'relu'),tf.keras.layers.Dropout(0.2),tf.keras.layers.GlobalAveragePooling2D(),tf.keras.layers.Dense(5, activation = 'softmax')])

In [15]:
model.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [16]:
epochs = 10
history = model.fit(train_generator, epochs = epochs, validation_data = val_generator)

Epoch 1/10
46/46 [==============================] - 97s 2s/step - loss: 0.6084 - accuracy: 0.7751 - val_loss: 0.3930 - val_accuracy: 0.8564
Epoch 2/10
46/46 [==============================] - 79s 2s/step - loss: 0.2647 - accuracy: 0.9040 - val_loss: 0.4088 - val_accuracy: 0.8550
Epoch 3/10
46/46 [==============================] - 88s 2s/step - loss: 0.1570 - accuracy: 0.9462 - val_loss: 0.4129 - val_accuracy: 0.8605
Epoch 4/10
46/46 [==============================] - 96s 2s/step - loss: 0.0961 - accuracy: 0.9701 - val_loss: 0.4356 - val_accuracy: 0.8564
Epoch 5/10
46/46 [==============================] - 100s 2s/step - loss: 0.0558 - accuracy: 0.9850 - val_loss: 0.4774 - val_accuracy: 0.8591
Epoch 6/10
46/46 [==============================] - 102s 2s/step - loss: 0.0358 - accuracy: 0.9939 - val_loss: 0.4841 - val_accuracy: 0.8605
Epoch 7/10
46/46 [==============================] - 104s 2s/step - loss: 0.0209 - accuracy: 0.9983 - val_loss: 0.4822 - val_accuracy: 0.8700
Epoch 8/10
46/46 

In [17]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
with open('model.tflite','wb') as f:
    f.write(tflite_model)

D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets
